# OpenAI Embedding For Similarity Search

In [1]:
!pip install pandas psycopg
import psycopg
import pandas as pd

In [2]:
from getpass import getpass
DATABASE_URL = getpass()

 ········


In [3]:
rows = []
async with await psycopg.AsyncConnection.connect(DATABASE_URL) as conn:
    async with conn.cursor() as cur:
        await cur.execute("""
            SELECT 
                store_id, 
                store_name, 
                brand, 
                address,
                ST_X(coordinates::geometry) AS longitude,
                ST_Y(coordinates::geometry) AS latitude
            FROM
                stores 
            WHERE 
                brand = '7-11' 
            AND
                address LIKE '%新北市%'
        """)

        async for row in cur:
            rows.append(row)
            
        await conn.commit()

pd.DataFrame(rows, columns=["store_id", "store_name", "brand", "address", "longitude", "latitude"])

,store_id,store_name,brand,address,longitude,latitude
0,211413,丁煌,7-11,新北市板橋區文化路二段227號、229號1樓,121.470813,25.028461
1,872441,千群,7-11,新北市板橋區東丘里民享街53號,121.475960,25.015124
2,243937,大合宜,7-11,新北市板橋區合宜路173號1樓,121.447108,24.998381
3,216304,大庭,7-11,新北市板橋區國光里6鄰國光路65號1樓,121.459051,25.018034
4,992037,中一,7-11,新北市板橋區中正路332之5號1樓,121.454566,25.022579
...,...,...,...,...,...,...
1189,253509,新翡翠灣,7-11,新北市萬里區龜吼里美崙90號,121.685859,25.185052
1190,153331,溫泉,7-11,新北市萬里區大鵬里萬里加投15鄰197號1樓,121.644145,25.215003
1191,276443,萬里,7-11,新北市萬里區瑪鋉路49號51號,121.687775,25.176091
1192,910552,那魯灣,7-11,新北市烏來區烏來里溫泉街80號,121.550714,24.861970


## Generate Embedding from Store information

```
openai({ store_id, store_name, brand, address, longitude, latitude }) => embedding
```



In [4]:
import json

records = pd.DataFrame(rows, columns=["store_id", "store_name", "brand", "address", "longitude", "latitude"])
records['raw_embedding'] = records.apply(lambda row: json.dumps(row.to_dict(), ensure_ascii=False).replace(" ", ""), axis=1)

records

,store_id,store_name,brand,address,longitude,latitude,raw_embedding
0,211413,丁煌,7-11,新北市板橋區文化路二段227號、229號1樓,121.470813,25.028461,"{""store_id"":""211413"",""store_name"":""丁煌"",""brand""..."
1,872441,千群,7-11,新北市板橋區東丘里民享街53號,121.475960,25.015124,"{""store_id"":""872441"",""store_name"":""千群"",""brand""..."
2,243937,大合宜,7-11,新北市板橋區合宜路173號1樓,121.447108,24.998381,"{""store_id"":""243937"",""store_name"":""大合宜"",""brand..."
3,216304,大庭,7-11,新北市板橋區國光里6鄰國光路65號1樓,121.459051,25.018034,"{""store_id"":""216304"",""store_name"":""大庭"",""brand""..."
4,992037,中一,7-11,新北市板橋區中正路332之5號1樓,121.454566,25.022579,"{""store_id"":""992037"",""store_name"":""中一"",""brand""..."
...,...,...,...,...,...,...,...
1189,253509,新翡翠灣,7-11,新北市萬里區龜吼里美崙90號,121.685859,25.185052,"{""store_id"":""253509"",""store_name"":""新翡翠灣"",""bran..."
1190,153331,溫泉,7-11,新北市萬里區大鵬里萬里加投15鄰197號1樓,121.644145,25.215003,"{""store_id"":""153331"",""store_name"":""溫泉"",""brand""..."
1191,276443,萬里,7-11,新北市萬里區瑪鋉路49號51號,121.687775,25.176091,"{""store_id"":""276443"",""store_name"":""萬里"",""brand""..."
1192,910552,那魯灣,7-11,新北市烏來區烏來里溫泉街80號,121.550714,24.861970,"{""store_id"":""910552"",""store_name"":""那魯灣"",""brand..."


In [5]:
!pip install openai

In [6]:
from getpass import getpass
OPENAI_API_KEY = getpass()

 ········


In [18]:
from openai import AsyncClient
client = AsyncClient(api_key=OPENAI_API_KEY)

res = await client.embeddings.create(
    input=records['raw_embedding'].tolist(),
    model="text-embedding-3-small",
    dimensions=512
)

records['embedding'] = [record.embedding for record in res.data]

records

,store_id,store_name,brand,address,longitude,latitude,raw_embedding,embedding
0,211413,丁煌,7-11,新北市板橋區文化路二段227號、229號1樓,121.470813,25.028461,"{""store_id"":""211413"",""store_name"":""丁煌"",""brand""...","[0.004629910457879305, -0.009368228726089, 0.0..."
1,872441,千群,7-11,新北市板橋區東丘里民享街53號,121.475960,25.015124,"{""store_id"":""872441"",""store_name"":""千群"",""brand""...","[-0.0005394704639911652, -0.04355490580201149,..."
2,243937,大合宜,7-11,新北市板橋區合宜路173號1樓,121.447108,24.998381,"{""store_id"":""243937"",""store_name"":""大合宜"",""brand...","[-0.019657833501696587, -0.007021873723715544,..."
3,216304,大庭,7-11,新北市板橋區國光里6鄰國光路65號1樓,121.459051,25.018034,"{""store_id"":""216304"",""store_name"":""大庭"",""brand""...","[0.010428563691675663, 0.009344775229692459, 0..."
4,992037,中一,7-11,新北市板橋區中正路332之5號1樓,121.454566,25.022579,"{""store_id"":""992037"",""store_name"":""中一"",""brand""...","[-0.02305421605706215, -0.008627786301076412, ..."
...,...,...,...,...,...,...,...,...
1189,253509,新翡翠灣,7-11,新北市萬里區龜吼里美崙90號,121.685859,25.185052,"{""store_id"":""253509"",""store_name"":""新翡翠灣"",""bran...","[-0.04615270718932152, -0.02944859489798546, 0..."
1190,153331,溫泉,7-11,新北市萬里區大鵬里萬里加投15鄰197號1樓,121.644145,25.215003,"{""store_id"":""153331"",""store_name"":""溫泉"",""brand""...","[-0.04377853125333786, 0.014506123960018158, 0..."
1191,276443,萬里,7-11,新北市萬里區瑪鋉路49號51號,121.687775,25.176091,"{""store_id"":""276443"",""store_name"":""萬里"",""brand""...","[-0.05004030838608742, -0.003377455286681652, ..."
1192,910552,那魯灣,7-11,新北市烏來區烏來里溫泉街80號,121.550714,24.861970,"{""store_id"":""910552"",""store_name"":""那魯灣"",""brand...","[-0.03233647346496582, -0.01807759515941143, 0..."


## Store and Indexing embedding on Database

In [19]:
!pip install pgvector

In [21]:
from pgvector.psycopg import register_vector_async

async with await psycopg.AsyncConnection.connect(DATABASE_URL) as conn:
    # support vector
    await register_vector_async(conn)
    
    async with conn.cursor() as cur:
        # initialize the pgvector and table for embeddings
        await cur.execute("""
            -- Enable the vector extension (if not already enabled)
            CREATE EXTENSION IF NOT EXISTS vector;
            
            -- Create the stores_embeddings table (if it does not already exist)
            CREATE TABLE IF NOT EXISTS stores_embeddings (
                store_id VARCHAR(6) NOT NULL,
                brand VARCHAR(10) NOT NULL CHECK (brand IN ('7-11', 'FamilyMart')),
                embedding VECTOR(512) NOT NULL,
                raw_embedding TEXT NOT NULL,
                FOREIGN KEY (store_id, brand) REFERENCES stores(store_id, brand) ON DELETE CASCADE,
                PRIMARY KEY (store_id, brand)
            );
            """
        )

        # insert the embeddings into table
        await cur.executemany("""
            INSERT INTO stores_embeddings 
                (store_id, brand, embedding, raw_embedding)
            VALUES 
                (%(store_id)s, %(brand)s, %(embedding)s, %(raw_embedding)s)
            ON CONFLICT 
                (store_id, brand) 
            DO UPDATE
            SET 
                store_id = EXCLUDED.store_id, 
                brand = EXCLUDED.brand, 
                embedding = EXCLUDED.embedding, 
                raw_embedding = EXCLUDED.raw_embedding
            ;
            """,
            records.to_dict('records')
        )
            
        await conn.commit()

        print('completed')

completed


## Similarity Search and Prompt Engineering

User input a query -> openai(query) -> embedding

similarity search(embedding) -> nearby neighbors

In [23]:
query = input()

 板橋新府


In [31]:
prompt = f'''
search similarity order by
1. store name like {query}
2. address like {query}
3. store nearby {query}
'''

res = await client.embeddings.create(
    input=prompt,
    model="text-embedding-3-small",
    dimensions=512
)

embedding = str(res.data[0].embedding)

In [32]:
from pgvector.psycopg import register_vector_async

rows = []
async with await psycopg.AsyncConnection.connect(DATABASE_URL) as conn:
    # support vector
    await register_vector_async(conn)
    
    async with conn.cursor() as cur:
        await cur.execute("""
            SELECT 
                s.store_id,
                s.store_name,
                s.address,
                ST_X(s.coordinates::geometry) AS longitude,
                ST_Y(s.coordinates::geometry) AS latitude,
                s.brand
            FROM 
                stores_embeddings e
            JOIN 
                stores s 
            ON 
                e.store_id = s.store_id AND e.brand = s.brand
            ORDER BY 
                e.embedding <-> %s
            LIMIT 
                10
            ;
            """,
            (embedding,)
        )

        async for row in cur:
            rows.append(row)
            
        await conn.commit()

pd.DataFrame(rows, columns=["store_id", "store_name", "brand", "address", "longitude", "latitude"])

,store_id,store_name,brand,address,longitude,latitude
0,228260,新府,新北市板橋區新府路75號B2樓,121.462501,25.013232,7-11
1,118349,新板橋,新北市板橋區文化路一段135號,121.462031,25.015210,7-11
2,237943,板府,新北市板橋區中山路一段158巷4號6號1樓,121.466059,25.011694,7-11
3,138549,二橋,新北市鶯歌區中正三路93號,121.338269,24.940923,7-11
4,257475,板橋金門,新北市板橋區金門街415號417號419號1樓,121.427373,24.983901,7-11
5,243236,板樂,新北市板橋區三民路二段35巷11號1樓,121.480180,25.017901,7-11
6,244387,板和,新北市板橋區三民路二段145號145-1號153巷1號,121.479291,25.015134,7-11
7,257590,板橋中正,新北市板橋區中正路379巷3弄1號3號,121.453391,25.021186,7-11
8,255066,板中,新北市板橋區府後街4.4-1.4-2號1樓,121.459144,25.010224,7-11
9,265225,松林,新北市板橋區松江街109巷1號松江街107號,121.469024,25.030554,7-11
